In [7]:
import morph_kgc

file_path = "data/outsiders_2021.parquet"
rml = "rml-mappings/mapping.ttl"

# configuration file
config_morph = """
                            [DataSourcePARQUET]
                            file_path=""" + file_path + """
                            mappings=""" + rml

# generate the triples and load them to an RDFlib graph
graph = morph_kgc.materialize(config_morph)
# work with the graph
#graph.serialize(destination='results/parquet.nt', format='nt')

INFO | 2022-06-23 10:54:34,265 | 256 mapping rules retrieved.
INFO | 2022-06-23 10:54:34,324 | Mapping partition with 1 groups generated.
INFO | 2022-06-23 10:54:34,324 | Maximum number of rules within mapping group: 256.
INFO | 2022-06-23 10:54:34,330 | Mappings processed in 0.882 seconds.


ArrowInvalid: No match for FieldRef.Name(awards/id) in summary: string
title: string
updated: timestamp[us, tz=UTC]
Número de Expediente: string
Estado: string
Nombre: string
Ubicación orgánica: string
Objeto del Contrato: string
Tipo de Contrato: double
Valor estimado del contrato: double
Presupuesto base sin impuestos: double
Clasificación CPV: list<item: double>
Código de Subentidad Nacional: string
Plazo de Ejecución (Duración): double
Tipo de Procedimiento: double
Presentación de Oferta (Fecha): string
Presentación de Oferta (Hora): string
Tipo de Anuncio: list<item: string>
Medio de Publicación: list<item: string>
Fecha de Publicación: list<item: string>
Plazo de Ejecución (Comienzo): string
Plazo de Ejecución (Fin): string
ContractFolderStatus - LocatedContractingParty - ParentLocatedParty - ParentLocatedParty - PartyName - Name: string
Pliego de cláusulas administrativas: string
Pliego de cláusulas administrativas (URI): string
Pliego de Prescripciones técnicas: string
Pliego d

In [5]:
import morph_kgc

file_path = "data/2019-01-01/ocds-0c46vo-0009-DN379620-1-tender-release.xml"
rml = "rml-mappings/mapping.ttl"

# configuration file
config_morph = """
            [DataSourceXML]
            file_path=""" + file_path + """
            mappings=""" + rml

# generate the triples and load them to an RDFlib graph
graph = morph_kgc.materialize(config_morph)
# work with the graph
graph.serialize(destination='results/xml.nt', format='nt')

INFO | 2022-06-23 10:17:48,200 | 256 mapping rules retrieved.
INFO | 2022-06-23 10:17:48,254 | Mapping partition with 1 groups generated.
INFO | 2022-06-23 10:17:48,255 | Maximum number of rules within mapping group: 256.
INFO | 2022-06-23 10:17:48,262 | Mappings processed in 0.799 seconds.
INFO | 2022-06-23 10:17:50,294 | Number of triples generated in total: 61.
c:\Users\Pedro\AppData\Local\Programs\Python\Python39\lib\site-packages\rdflib\plugins\serializers\nt.py:36: UserWarning: NTSerializer always uses UTF-8 encoding. Given encoding was: None
  warnings.warn(


<Graph identifier=Nd8602e3bd87f4f769f42fe59406d6dd7 (<class 'rdflib.graph.Graph'>)>